In [3]:
#Resize image to desired dimension
import cv2
image = cv2.imread("/content/drive/MyDrive/sample_dog.jpg", 1)
resized_image = cv2.resize(image, (512, 512))
cv2.imwrite("/content/drive/MyDrive/resized_dog.jpg", resized_image)

True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import torch
import torch.nn as nn

class MLP(nn.Module):
  def __init__(self, num_inputs):
    super(MLP, self).__init__()
    self.fcn1 = nn.Linear(num_inputs, 256)
    self.fcn2 = nn.Linear(256, 256)
    self.fcn4 = nn.Linear(256, 3)
    self.relu = nn.ReLU()
    self.sig = nn.Sigmoid()

  def forward(self, X):
    X = self.relu(self.fcn1(X))
    X = self.fc2_bn(X)
    X = self.relu(self.fcn2(X))
    X = self.relu(self.fcn2(X))
    X = self.relu(self.fcn2(X))
    X = self.relu(self.fcn2(X))
    X = self.sig(self.fcn4(X))
    return X


In [5]:
import os
import pandas as pd
import numpy as np
import torchvision
from torch.utils.data import Dataset
import torchvision.transforms as T
from PIL import Image

def input_mapping(x, B):
    if B is None:
        return x
    else:
        B = B.to(x.device)
        x_proj = (2.*np.pi*x) @ B.T
        return torch.cat([torch.sin(x_proj), torch.cos(x_proj)], dim=-1)

class CustomDataset(Dataset):
    def __init__(self, n, img_path, B_gauss):
        self.X = []
        self.Y = []
        self.B_gauss = B_gauss
        
        image = torchvision.io.read_image(img_path)
        _, height, width = image.shape
        image = image.numpy()
        
        for i in range(height):
          for j in range(width):
            self.X.append(np.array([i, j])/255.0)
            self.Y.append(image[:, i, j]/255.0)
        self.Y = [l.tolist() for l in self.Y]
        self.X = torch.FloatTensor(self.X).to(device = device)
        self.X = input_mapping(self.X, self.B_gauss)

        if n != None:
          self.X = self.X[:n]
          self.Y = self.Y[:n]

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
      idx_sample = self.X[idx]
      idx_target = self.Y[idx]
      return {
          "x": torch.tensor(idx_sample, dtype = torch.float),
          "y": torch.tensor(idx_target, dtype = torch.float),
      }
  

In [6]:
dump_path = "/content/drive/MyDrive/dog_dump"
img_path = "/content/drive/MyDrive/resized_dog.jpg"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

mapping_size = 256
B_gauss = torch.normal(0, 1, size=(mapping_size,2))
B_gauss = B_gauss*10

dataset = CustomDataset(None, img_path, B_gauss)
train_loader  = torch.utils.data.DataLoader(dataset, batch_size = 512)

X = []
image = torchvision.io.read_image(img_path)
_, height, width = image.shape
image = image.numpy()
for i in range(height):
  for j in range(width):
    X.append(np.array([i, j])/255.0)
picture = torch.FloatTensor(X).to(device = device)
picture = input_mapping(picture, B_gauss)



num_epochs = 500
losses = []
learning_rate = 0.0001
model = MLP(mapping_size*2)
model = model.to(device=device)
model.train()
criterian = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
for epoch in range(1, num_epochs+1):
  for data in train_loader:
    xtrain = data["x"].to(device = device)
    ytrain = data["y"].to(device = device)
    pred = model(xtrain)
    loss = criterian(pred, ytrain)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  losses.append(loss.item())
  if epoch%5==0:
    with torch.no_grad():
      picture_pred = model(picture)
      picture_pred = picture_pred*255
      picture_pred = torch.reshape(picture_pred, (512, 512, 3))
      res = picture_pred.detach().cpu().numpy()
      res = res.astype(np.uint8)
      res = Image.fromarray(res, 'RGB')
      res.save(f"{dump_path}/2000_nobatchnorm_enc256_pred_dog_512_deeper4_gauss10_{epoch}.jpg")
  print(f"epoch:{epoch}, loss = {loss.item():.4f}")


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


epoch:1, loss = 0.0042
epoch:2, loss = 0.0041
epoch:3, loss = 0.0038
epoch:4, loss = 0.0039
epoch:5, loss = 0.0035
epoch:6, loss = 0.0036
epoch:7, loss = 0.0035
epoch:8, loss = 0.0032
epoch:9, loss = 0.0028
epoch:10, loss = 0.0027
epoch:11, loss = 0.0027
epoch:12, loss = 0.0024
epoch:13, loss = 0.0025
epoch:14, loss = 0.0026
epoch:15, loss = 0.0024
epoch:16, loss = 0.0023
epoch:17, loss = 0.0029
epoch:18, loss = 0.0033
epoch:19, loss = 0.0034
epoch:20, loss = 0.0028
epoch:21, loss = 0.0039
epoch:22, loss = 0.0032
epoch:23, loss = 0.0028
epoch:24, loss = 0.0026
epoch:25, loss = 0.0033
epoch:26, loss = 0.0025
epoch:27, loss = 0.0024
epoch:28, loss = 0.0021
epoch:29, loss = 0.0022
epoch:30, loss = 0.0018
epoch:31, loss = 0.0020
epoch:32, loss = 0.0017
epoch:33, loss = 0.0018
epoch:34, loss = 0.0019
epoch:35, loss = 0.0014
epoch:36, loss = 0.0016
epoch:37, loss = 0.0018
epoch:38, loss = 0.0018
epoch:39, loss = 0.0019
epoch:40, loss = 0.0015
epoch:41, loss = 0.0013
epoch:42, loss = 0.0013
e

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = range(1, num_epochs+1)
y = losses
plt.plot(x, y)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()
